In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
#from keras.layers import LSTM, Dropout, Dense, Layer, Conv1D
from keras.models import Sequential


2023-02-24 19:41:46.102441: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-02-24 19:41:46.255882: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-02-24 19:41:46.255902: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2023-02-24 19:41:46.290337: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-02-24 19:41:47.160369: W tensorflow/stream_executor/platform/de

In [2]:
import os
os.chdir("..")
from src import *


In [3]:
_df = pd.read_csv("Data/wig20_d.csv")
# df = pd.read_csv("Data/mwig40_d.csv")
# df = pd.read_csv("Data/swig80_d.csv")

_df.drop(range(1000), inplace=True)
_df.reset_index(inplace=True, drop=True)
_df.head()

,Data,Otwarcie,Najwyzszy,Najnizszy,Zamkniecie,Wolumen
0,1997-02-06,1717.2,1779.8,1717.2,1779.8,4435886.0
1,1997-02-07,1778.2,1808.7,1778.2,1799.0,5448243.0
2,1997-02-10,1797.4,1797.4,1780.3,1783.3,6513315.0
3,1997-02-11,1803.3,1832.2,1803.3,1832.0,5146340.0
4,1997-02-12,1824.6,1824.6,1804.9,1810.2,5748398.0


In [4]:
_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6429 entries, 0 to 6428
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Data        6429 non-null   object 
 1   Otwarcie    6429 non-null   float64
 2   Najwyzszy   6429 non-null   float64
 3   Najnizszy   6429 non-null   float64
 4   Zamkniecie  6429 non-null   float64
 5   Wolumen     6429 non-null   float64
dtypes: float64(5), object(1)
memory usage: 301.5+ KB


In [5]:
_df.columns

Index(['Data', 'Otwarcie', 'Najwyzszy', 'Najnizszy', 'Zamkniecie', 'Wolumen'], dtype='object')

In [6]:
data = _df.drop(columns=["Data"]).values.transpose()

In [7]:
data.shape

(5, 6429)

In [8]:
_x_train, _y_train, _x_test, _y_test, _x_validation, _y_validation = data_prepare_to_model_2d(data, test_ratio=0.2, validation_ratio=0.2, batch_size=32)

train
test
validation


In [9]:
print("train", _x_train.shape)
print("test", _x_test.shape)
print("validation", _x_validation.shape)


train (3825, 5, 32, 2)
test (1254, 5, 32, 2)
validation (1254, 5, 32, 2)


In [10]:
print("train", _y_train.shape)
print("test", _y_test.shape)
print("validation", _y_validation.shape)


train (3825, 5, 1, 2)
test (1254, 5, 1, 2)
validation (1254, 5, 1, 2)


In [11]:
model = tf.keras.Sequential()

model.add(tf.keras.Input(shape=_x_train.shape[1:]))
for i in range(7):
    model.add(tf.keras.layers.Conv2D(2, (1, 3), 1, padding="same"))
    model.add(tf.keras.layers.MaxPooling2D(pool_size=(1, 3), strides=(1, 3), padding='same'))

model.add(tf.keras.layers.Dense(12))
model.add(tf.keras.layers.Dense(7))
model.add(tf.keras.layers.Dense(3, activation="softmax"))
model.add(tf.keras.layers.Dense(2))


2023-02-24 19:43:31.767251: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2023-02-24 19:43:31.767287: W tensorflow/stream_executor/cuda/cuda_driver.cc:263] failed call to cuInit: UNKNOWN ERROR (303)
2023-02-24 19:43:31.767319: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (banjaro): /proc/driver/nvidia/version does not exist
2023-02-24 19:43:31.768386: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [12]:
model.build(input_shape=_x_train.shape[1:])

In [13]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 5, 32, 2)          14        
                                                                 
 max_pooling2d (MaxPooling2D  (None, 5, 11, 2)         0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 5, 11, 2)          14        
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 5, 4, 2)          0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 5, 4, 2)           14        
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 5, 2, 2)          0

In [12]:
# model = tf.keras.Sequential()
#
# for i in range(7):
#     model.add(Cdt1dLayer())
#     model.add(tf.keras.layers.MaxPooling2D(pool_size=(1, 3), strides=(1, 3), padding='same'))
#
# model.add(tf.keras.layers.Dense(12))
# model.add(tf.keras.layers.Dense(7))
# model.add(tf.keras.layers.Dense(3, activation="softmax"))
# model.add(tf.keras.layers.Dense(2))


In [13]:
# model.build(input_shape=_x_train.shape)

In [14]:
# model.summary()

In [14]:
# model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3),
#               loss=tf.keras.losses.BinaryCrossentropy())
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3),
              loss='mean_squared_error')


In [15]:
print(_x_train.shape, _y_train.shape)

(3825, 5, 32, 2) (3825, 5, 1, 2)


In [17]:
# b_size = 32
# model.fit(_x_train[:b_size, :, :, :], _y_train[:b_size, :, :, :])

In [16]:
model.fit(_x_train, _y_train, epochs=10)

Epoch 1/10
120/120 [==============================] - 1s 4ms/step - loss: 140415156092928.0000
Epoch 2/10
120/120 [==============================] - 1s 4ms/step - loss: 140415164481536.0000
Epoch 3/10
120/120 [==============================] - 1s 4ms/step - loss: 140415172870144.0000
Epoch 4/10
120/120 [==============================] - 1s 5ms/step - loss: 140415206424576.0000
Epoch 5/10
120/120 [==============================] - 1s 4ms/step - loss: 140415139315712.0000
Epoch 6/10
120/120 [==============================] - 1s 5ms/step - loss: 140415156092928.0000
Epoch 7/10
120/120 [==============================] - 1s 4ms/step - loss: 140415172870144.0000
Epoch 8/10
120/120 [==============================] - 1s 4ms/step - loss: 140415139315712.0000
Epoch 9/10
120/120 [==============================] - 0s 4ms/step - loss: 140415156092928.0000
Epoch 10/10
120/120 [==============================] - 0s 4ms/step - loss: 140415147704320.0000


In [18]:
model.evaluate(_x_test, _y_test)

40/40 [==============================] - 0s 3ms/step - loss: 190619188199424.0000


190619188199424.0

In [19]:
model.save("Models/cdt_2")

INFO:tensorflow:Assets written to: Models/cdt_2/assets


INFO:tensorflow:Assets written to: Models/cdt_2/assets


In [20]:
_x_train

array([[[[1.7172000e+03, 1.7172000e+03],
         [1.7782000e+03, 1.7782000e+03],
         [1.7974000e+03, 1.7974000e+03],
         ...,
         [2.3735000e+03, 2.3735000e+03],
         [2.2916599e+03, 2.2916599e+03],
         [2.2837200e+03, 2.2837200e+03]],

        [[1.7798000e+03, 1.7798000e+03],
         [1.8087000e+03, 1.8087000e+03],
         [1.7974000e+03, 1.7974000e+03],
         ...,
         [2.3801001e+03, 2.3801001e+03],
         [2.3078601e+03, 2.3078601e+03],
         [2.3203701e+03, 2.3203701e+03]],

        [[1.7172000e+03, 1.7172000e+03],
         [1.7782000e+03, 1.7782000e+03],
         [1.7803000e+03, 1.7803000e+03],
         ...,
         [2.3103899e+03, 2.3103899e+03],
         [2.2851599e+03, 2.2851599e+03],
         [2.2734900e+03, 2.2734900e+03]],

        [[1.7798000e+03, 1.7798000e+03],
         [1.7990000e+03, 1.7990000e+03],
         [1.7833000e+03, 1.7833000e+03],
         ...,
         [2.3402400e+03, 2.3402400e+03],
         [2.3025300e+03, 2.3025300e+

In [3]:
res = Cdt1dLayer()(
    np.broadcast_to(
        np.moveaxis(
            np.arange(20).reshape([1, 1, 4, 5]).astype("float32"),
            0, 3),
        [1, 4, 5, 2]
    )
)

2022-12-20 23:05:10.978727: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-12-20 23:05:10.978749: W tensorflow/stream_executor/cuda/cuda_driver.cc:263] failed call to cuInit: UNKNOWN ERROR (303)
2022-12-20 23:05:10.978766: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (banjaro): /proc/driver/nvidia/version does not exist
2022-12-20 23:05:10.978956: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [4]:
res.shape

(1, 4, 5, 2)

In [5]:
res[0,:,:,0]

array([[ 1.,  3.,  6.,  9.,  7.],
       [11., 18., 21., 24., 17.],
       [21., 33., 36., 39., 27.],
       [31., 48., 51., 54., 37.]])

In [6]:
res[0,:,:,1]

array([[ 0.5,  1.5,  3. ,  4.5,  3.5],
       [ 5.5,  9. , 10.5, 12. ,  8.5],
       [10.5, 16.5, 18. , 19.5, 13.5],
       [15.5, 24. , 25.5, 27. , 18.5]])

In [3]:
model = tf.keras.Sequential()

for i in range(7):
    model.add(Cdt1dLayer())
    model.add(tf.keras.layers.MaxPooling1D(pool_size=3, strides=3, padding='same'))

model.add(tf.keras.layers.Dense(12))
model.add(tf.keras.layers.Dense(7))
model.add(tf.keras.layers.Dense(3))


2022-12-23 19:47:15.601358: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-12-23 19:47:15.601387: W tensorflow/stream_executor/cuda/cuda_driver.cc:263] failed call to cuInit: UNKNOWN ERROR (303)
2022-12-23 19:47:15.601412: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (banjaro): /proc/driver/nvidia/version does not exist
2022-12-23 19:47:15.601707: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [4]:
model.build(input_shape=(1, 5, 1600, 2))

AttributeError: Exception encountered when calling layer "cdt1d_layer" "                 f"(type Cdt1dLayer).

in user code:

    File "/home/bartko/Repozytoria/inzynierka/src/CDT_1D_CNN/cdt_1d_layer.py", line 69, in call  *
        inputs = inputs.numpy()

    AttributeError: 'Tensor' object has no attribute 'numpy'


Call arguments received by layer "cdt1d_layer" "                 f"(type Cdt1dLayer):
  • inputs=tf.Tensor(shape=(1, 5, 1600, 2), dtype=float32)
  • kwargs={'training': 'None'}

In [ ]:
model.summary()

AttributeError: module 'tensorflow' has no attribute 'enable_eager_execution'

In [13]:
input_shape = (10,20,)

model = keras.Sequential()
#model.add(tf.keras.Input(shape=(10, None,)))

model.add(Cdt1dLayer(input_shape=input_shape))
model.add(tf.keras.layers.MaxPooling1D(pool_size=3, strides=3, padding='same'))

for i in range(6):
    model.add(Cdt1dLayer())
    model.add(tf.keras.layers.MaxPooling1D(pool_size=3, strides=3, padding='same'))

model.add(tf.keras.layers.Dense(12))
model.add(tf.keras.layers.Dense(7))
model.add(tf.keras.layers.Dense(3, activation="softmax"))

model.build(input_shape)

ValueError: Exception encountered when calling layer "module_wrapper_28" "                 f"(type ModuleWrapper).

in user code:

    File "/home/bartko/Repozytoria/inzynierka/src/CDT_1D_CNN/cdt_1d_layer.py", line 57, in call  *
        inputs_flat = inputs_flat_with_pad(inputs)
    File "/home/bartko/Repozytoria/inzynierka/src/CDT_1D_CNN/cdt_1d_layer.py", line 87, in inputs_flat_with_pad  *
        dim_batches, dim_y, dim_x = inputs.shape

    ValueError: not enough values to unpack (expected 3, got 2)


Call arguments received by layer "module_wrapper_28" "                 f"(type ModuleWrapper):
  • args=('tf.Tensor(shape=(10, 20), dtype=float32)',)
  • kwargs={'training': 'None'}

In [12]:
model.summary()

ValueError: This model has not yet been built. Build the model first by calling `build()` or by calling the model on a batch of data.

In [18]:
res_mp = tf.keras.layers.MaxPooling1D(pool_size=3, strides=3, padding='same')(res[:, :, :]).numpy()


In [ ]:
model = Sequential()

model.add(LSTM(units=50, return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(units=50, return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(units=50, return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(units=50))
model.add(Dropout(0.2))
model.add(Dense(units=y_train.shape[1]))

model.compile(optimizer='adam', loss='mean_squared_error')
model.fit(X_train, y_train, epochs=60, batch_size=32)